In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import logging
import asyncio
from fastmcp import Client
config = {
    "mcpServers": {
        "opgg": {"command": "npx",
        "args":[
            "-y",
            "supergateway",
            "--streamableHttp",
            "https://mcp-api.op.gg/mcp"
        ]}
    }
}

client = Client(config)

# Tool(name='lol_list_champion_leaderboard', title=None, description='Retrieves the leaderboard of master players who play a specific champion well. You can also look up the high ranked players, like the high winrate 5 players in #1-#30, to retrieve their champion performance or recent match history to teach how to play the champions.', inputSchema={'type': 'object', 'properties': {'region': {'type': 'string', 'enum': ['KR', 'BR', 'EUNE', 'EUW', 'LAN', 'LAS', 'NA', 'OCE', 'RU', 'TR', 'JP', 'PH', 'SG', 'TH', 'TW', 'VN', 'ME', 'SEA'], 'description': 'Regions in the game League of Legends.'}, 'champion': {'type': 'string', 'enum': ['ANNIE', 'OLAF', 'GALIO', 'TWISTED_FATE', 'XIN_ZHAO', 'URGOT', 'LEBLANC', 'VLADIMIR', 'FIDDLESTICKS', 'KAYLE', 'MASTER_YI', 'ALISTAR', 'RYZE', 'SION

async with client:
    tool_list = await client.list_tools()
    for tool in tool_list:
        print(f"Tool: {tool.name} ")
        print(f" Description: {tool.description}")
        for k in tool.inputSchema["properties"]:
            print(f"  - {k}: {tool.inputSchema["properties"][k]}")

        print("\n")

CancelledNotification.method
  Input should be 'notifications/cancelled' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
CancelledNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
ProgressNotification.method
  Input should be 'notifications/progress' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
ProgressNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
LoggingMessageNotification.method
  Input should be 'notifications/message' [type=literal_err

Tool: lol_list_champion_leaderboard 
 Description: Retrieves the leaderboard of master players who play a specific champion well. You can also look up the high ranked players, like the high winrate 5 players in #1-#30, to retrieve their champion performance or recent match history to teach how to play the champions.
  - region: {'type': 'string', 'enum': ['KR', 'BR', 'EUNE', 'EUW', 'LAN', 'LAS', 'NA', 'OCE', 'RU', 'TR', 'JP', 'PH', 'SG', 'TH', 'TW', 'VN', 'ME', 'SEA'], 'description': 'Regions in the game League of Legends.'}
  - champion: {'type': 'string', 'enum': ['ANNIE', 'OLAF', 'GALIO', 'TWISTED_FATE', 'XIN_ZHAO', 'URGOT', 'LEBLANC', 'VLADIMIR', 'FIDDLESTICKS', 'KAYLE', 'MASTER_YI', 'ALISTAR', 'RYZE', 'SION', 'SIVIR', 'SORAKA', 'TEEMO', 'TRISTANA', 'WARWICK', 'NUNU_WILLUMP', 'MISS_FORTUNE', 'ASHE', 'TRYNDAMERE', 'JAX', 'MORGANA', 'ZILEAN', 'SINGED', 'EVELYNN', 'TWITCH', 'KARTHUS', 'CHOGATH', 'AMUMU', 'RAMMUS', 'ANIVIA', 'SHACO', 'DR_MUNDO', 'SONA', 'KASSADIN', 'IRELIA', 'JANNA', '

CancelledNotification.method
  Input should be 'notifications/cancelled' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
CancelledNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
ProgressNotification.method
  Input should be 'notifications/progress' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
ProgressNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
LoggingMessageNotification.method
  Input should be 'notifications/message' [type=literal_err

In [31]:
async with client:
    result = await client.call_tool("lol_list_champions", {
        "lang":"zh-TW"
    })
    result_json = result.content[0].text

    import json
    result_data = json.loads(result_json)
    print(result_data)
    with open("champions.json", "w", encoding="utf-8") as f:
        json.dump(result_data, f, ensure_ascii=False, indent=4)

{'column_descriptions': {'champion_id': 'Unique champion identifier', 'key': 'Champion key (internal identifier)', 'name': 'Champion name (localized)', 'release_date': 'Champion release date'}, 'data': {'champions': {'headers': ['champion_id', 'key', 'name', 'release_date'], 'rows': [[1, 'Annie', '安妮', '2010-07-13T09:00:00+09:00'], [2, 'Olaf', '歐拉夫', '2012-12-21T05:00:00+09:00'], [3, 'Galio', '加里歐', '2012-12-21T06:00:00+09:00'], [4, 'TwistedFate', '逆命', '2010-07-13T09:00:00+09:00'], [5, 'XinZhao', '趙信', '2012-12-21T06:00:00+09:00'], [6, 'Urgot', '烏爾加特', '2012-12-21T05:00:00+09:00'], [7, 'Leblanc', '勒布朗', '2012-12-21T05:00:00+09:00'], [8, 'Vladimir', '弗拉迪米爾', '2012-12-21T05:00:00+09:00'], [9, 'Fiddlesticks', '費德提克', '2010-07-13T09:00:00+09:00'], [10, 'Kayle', '凱爾', '2010-07-13T09:00:00+09:00'], [11, 'MasterYi', '易大師', '2010-07-13T09:00:00+09:00'], [12, 'Alistar', '亞歷斯塔', '2010-07-13T09:00:00+09:00'], [13, 'Ryze', '雷茲', '2010-07-13T09:00:00+09:00'], [14, 'Sion', '賽恩', '2010-07-13T09:00:0

In [22]:
async with client:
    result = await client.call_tool("lol_list_champion_details", {
        "champions":["AHRI"],
        "lang":"zh_TW",
        "desired_value_description":"damage"
    })
    result_json = result.content[0].text

    import json
    result_data = json.loads(result_json)
    print(result_data)

{'column_descriptions': {'id': 'Champion numeric ID', 'key': 'Champion key identifier', 'name': 'Champion display name', 'evolve': 'Champion evolution information', 'blurb': 'Short description of the champion', 'title': 'Champion title', 'tags': 'Champion tags/roles', 'lore': 'Full lore text', 'partype': 'Resource type (mana, energy, etc.)', 'info': 'Champion stats information', 'info.attack': 'Attack rating', 'info.defense': 'Defense rating', 'info.magic': 'Magic rating', 'info.difficulty': 'Difficulty rating', 'stats': 'Base statistics', 'stats.hp': 'Base health points', 'stats.hpperlevel': 'Health per level', 'stats.mp': 'Base mana points', 'stats.mpperlevel': 'Mana per level', 'stats.movespeed': 'Movement speed', 'stats.armor': 'Base armor', 'stats.armorperlevel': 'Armor per level', 'stats.spellblock': 'Base magic resist', 'stats.spellblockperlevel': 'Magic resist per level', 'stats.attackrange': 'Attack range', 'stats.hpregen': 'Health regeneration', 'stats.hpregenperlevel': 'Heal

In [28]:
async with client:
    result = await client.call_tool("lol_list_discounted_skins", {
        "lang":"zh_TW",
        "desired_value_description":"damage"
    })
    result_json = result.content[0].text

    import json
    result_data = json.loads(result_json)
    print(result_data)

{'column_descriptions': {'skin_sales.*.champion_id': 'Champion identifier', 'skin_sales.*.skin_id': 'Skin identifier', 'skin_sales.*.currency': 'Currency type (e.g., RP)', 'skin_sales.*.cost': 'Cost of the skin', 'skin_sales.*.discount_rate': 'Discount rate (e.g., 0.5 for 50% off)', 'skin_sales.*.started_at': 'Sale start date and time (ISO 8601 format)', 'skin_sales.*.ended_at': 'Sale end date and time (ISO 8601 format)'}, 'data': {'skin_sales': {'headers': ['champion_id', 'skin_id', 'currency', 'cost', 'discount_rate', 'started_at', 'ended_at'], 'rows': [[8, 8000, 'RP', 355, 0.55, '2025-10-20T08:00:00+09:00', '2025-10-27T08:00:00+09:00'], [15, 15034, 'RP', 607, 0.55, '2025-10-20T08:00:00+09:00', '2025-10-27T08:00:00+09:00'], [23, 23010, 'RP', 540, 0.6, '2025-10-20T08:00:00+09:00', '2025-10-27T08:00:00+09:00'], [32, 32006, 'RP', 337, 0.55, '2025-10-20T08:00:00+09:00', '2025-10-27T08:00:00+09:00'], [59, 59002, 'RP', 300, 0.6, '2025-10-20T08:00:00+09:00', '2025-10-27T08:00:00+09:00'], [7

In [25]:
async with client:
    result = await client.call_tool("lol_get_champion_synergies", {
        "champion":"AHRI",
        "my_position":"mid",
        "synergy_position":"jungle",
        "lang":"zh_TW",
        "desired_value_description":"damage"
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        print(result_data)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print("Response text:", result_json)

Failed to decode JSON
Response text: {"column_descriptions":{"champion":"Your champion","my_position":"Your position","synergy_position":"Target synergy position","champion_id":"Your champion ID","position":"Your position","synergy_champion_id":"Recommended synergy champion ID","score_rank":"Synergy ranking (1 is best)","score":"Synergy score","play":"Number of games played together","win":"Number of games won together","win_rate":"Win rate when played together","synergy_tier_data.tier":"Synergy tier (0=OP, 1=S, 2=A, 3=B, 4=C)","synergy_tier_data.rank":"Current rank within tier","synergy_tier_data.rank_prev":"Previous rank within tier","synergy_tier_data.rank_prev_patch":"Rank in previous patch"},"data":{"synergies":{"headers":["champion_id","position","synergy_champion_id","synergy_position","score_rank","score","play","win","win_rate","synergy_tier_data.tier","synergy_tier_data.rank","synergy_tier_data.rank_prev","synergy_tier_data.rank_prev_patch"],"rows":[[103,"MID",64,"JUNGLE",1,0

In [ ]:
async with client:
    result = await client.call_tool("lol_list_items", {
        "lang":"zh_TW",
        "map":"SUMMONERS_RIFT"
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        print(result_data)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)

{'column_descriptions': {'item_id': 'Unique item identifier', 'name': 'Item name (localized)', 'is_mythic': 'Whether the item is mythic', 'into_items': 'Item IDs this builds into (JSON array)', 'from_items': 'Item IDs this builds from (JSON array)', 'gold_sell': 'Gold received when selling', 'gold_total': 'Total gold cost', 'gold_base': 'Base gold cost', 'gold_purchasable': 'Whether item can be purchased', 'plaintext': 'Short description', 'depth': 'Item build depth (0 for base items)'}, 'data': {'items': {'headers': ['item_id', 'name', 'is_mythic', 'into_items', 'from_items', 'gold_sell', 'gold_total', 'gold_base', 'gold_purchasable', 'plaintext', 'depth'], 'rows': [[1001, '鞋子', False, '[3005,3047,3006,3009,3010,3020,3111,3117,3158]', '[]', 210, 300, 300, True, '稍微增加跑速', 0], [1004, '仙女護符', False, '[3114,3012,4642]', '[]', 140, 200, 200, True, '稍微增加魔力回復', 0], [1006, '治療寶珠', False, '[3109,3211,323109,3801]', '[]', 120, 300, 300, True, '稍微增加生命回復', 0], [1018, '靈巧披風', False, '[123430,66766

In [27]:
async with client:
    result = await client.call_tool("lol_list_lane_meta_champions", {
        "lang":"zh_TW",
        "position":"mid",
        
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        print(result_data)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)

{'column_descriptions': {'position.*.champion_id': 'Unique champion identifier', 'position.*.is_rip': 'Whether champion is considered RIP (weak/unplayable) in this position', 'position.*.play': 'Number of matches played in this position', 'position.*.win': 'Number of wins in this position', 'position.*.kill': 'Average kills in this position', 'position.*.win_rate': 'Win rate percentage in this position', 'position.*.pick_rate': 'Pick rate percentage in this position', 'position.*.role_rate': 'Percentage of games this champion plays in this role', 'position.*.ban_rate': 'Ban rate percentage for this position', 'position.*.kda': 'KDA ratio in this position', 'position.*.tier': 'Tier ranking (1=OP, 2=Strong, 3=Good, 4=Average, 5=Weak)', 'position.*.rank': 'Current rank in this position', 'position.*.rank_prev': 'Previous rank in this position', 'position.*.rank_prev_patch': 'Rank in previous patch for this position'}, 'data': {'position.mid': {'headers': ['champion_id', 'is_rip', 'play', 

In [ ]:
async with client:
    result = await client.call_tool("lol_list_lane_meta_champions", {
        "lang":"zh_TW",
        "position":"mid",
        
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        print(result_data)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)

In [ ]:
async with client:
    result = await client.call_tool("lol_get_champion_analysis", {
        "game_mode":"ranked",
        "champion": "AHRI",
        
        "lang":"zh_TW",
        "position":"mid",
        
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        with open("lol_get_champion_analysis.json", "w", encoding="utf-8") as f:
            json.dump(result_data, f, ensure_ascii=False, indent=4)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)

In [34]:
async with client:
    result = await client.call_tool("lol_get_pro_player_riot_id", {
        "player_name":"Faker",
        "region": "KR",
        "return_suggestions": True
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        with open("pro_riot_id.json", "w", encoding="utf-8") as f:
            json.dump(result_data, f, ensure_ascii=False, indent=4)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)

In [36]:
async with client:
    result = await client.call_tool("lol_get_lane_matchup_guide", {
        "lang":"zh-TW",
        "position":"mid",
        "my_champion":"AHRI",
        "opponent_champion":"ZED",
    })
    result_json = result.content[0].text

    import json
    try:
        result_data = json.loads(result_json)
        with open("lane_matchup_guide.json", "w", encoding="utf-8") as f:
            json.dump(result_data, f, ensure_ascii=False, indent=4)
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        print(result_json)